# Part 1 - Preprocessing

Scripts to show python and NLP

In [23]:
import re
import spacy
import html5lib
from bs4 import BeautifulSoup
from urllib import request
import nltk
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
import wordcloud as wc
from gensim.models.phrases import Phrases, Phraser
# from google.colab import drive

# drive.mount('/content/drive')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

# path = "/content/drive/MyDrive/_FACULDADE/7º Semestre/Tópicos VIII/_Final/V2/"
path = ""

# sentence tokenize object
sentences = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alisson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alisson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/alisson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alisson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /home/alisson/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Dealing with web pages

In [24]:
spacy.cli.download("en_core_web_md")  # en_core_web_sm en_core_web_lg
nlp = spacy.load('en_core_web_md')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [25]:
urlData1 = f"https://docs.google.com/spreadsheets/d/1sbp6Y1YZWHILHMma8qphe8tgJvDQ7s_ubLJhY2hSJI8/gviz/tq?tqx=out:csv&sheet=test"
urlData2 = f"https://docs.google.com/spreadsheets/d/1CSIVmE1NKiui-EEEHRnnsXzXmy908mQRtasSmtQe7x8/gviz/tq?tqx=out:csv&sheet=validation"

df1 = pd.read_csv(urlData1)
df2 = pd.read_csv(urlData2)

news = df1.append(df2, ignore_index=True)['article'][:1000]
news

0      Ever noticed how plane seats appear to be gett...
1      A drunk teenage boy had to be rescued by secur...
2      Dougie Freedman is on the verge of agreeing a ...
3      Liverpool target Neto is also wanted by PSG an...
4      Bruce Jenner will break his silence in a two-h...
                             ...                        
995    Women travelling on the London Underground are...
996    Millions of devout Christians around the world...
997    Floyd Mayweather and Manny Pacquiao may only b...
998    Nigel Farage suffered a fresh blow today after...
999    More than 140 children have been removed from ...
Name: article, Length: 1000, dtype: object

In [26]:
treatment = False
wordcloud = False
gridSearch = False

Removing Blacklist words:

In [27]:
if treatment:
  blackList = ["cnn", "bbc", "(CNN)", '\xad', "abc news", "\u200b", "\u202c",
                "\u202a#", "\u200e", "\u202c", "whyimnotvotingforhillary", "\"\'", "00pm", "042secs", "10", '000', '11', '111th', '112million', '117million', '12th', '130', '130million', '13th', '140billion', '140million', '147notjustanumber', '14billion', '14th', '15', '15am', '15th', '16th', '17', '177billion', '17th', '18', '1800', '180billion', '1850', '1870', '18s', '18th', '19', '1920', '1930', '1940', '1960', '1970', '1980', '1990', '19s', '19th', '20', '2000', '208', '20st', '20th', '21inch', '23rd', '24', '25st', '27', '27pm', '280sl', '2w', '30', '300_missile', '300er', '30am', '30pm', '30th', '31pm', '31st', '33minute', '35million', '363ad', '368million', '377million', '38', '385million', '390million', '39th', '3_contest', '3d', '3lbs', '405nm', '42million', '45min', '47', '49th', '4million', '506million', '50metre', '50pm', '50th', '52pm', '545million', '55s', '59', '5lbs', '5million', '617s', '61ad', '64th', '65th', '669million', '6l', '6lbs', '6music', '765million', '7million', '82pt', '84pt', '85pt', '8lbs', '8mmm', '8mmm_aboriginal', '90', '979pp', '9billion', '00', '09', '0kms', '3_contest', '45pm', '60s', '61', '7sqm']
  blackList = list(map(lambda a: "(?i)" + a, blackList))
  news.replace(blackList, "", inplace=True, regex=True)

  for i in range(len(news)):
      news[i] = re.sub(
          r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*", "", news[i])

  news

In [28]:
if treatment:
  dlemma=[]
  for d in news:
    tdoc=nlp(d)
    lm=[" ".join([token.lemma_.lower() for token in tdoc if 
      token.pos_ in ['NOUN','PROPN'] and 
      token.pos_ not in ['TIME', '-PRON-', 'CARDINAL', 'ORDINAL'] and
      token.is_stop == False and 
      token.is_punct == False and 
      token.is_digit == False and 
      len(token.lemma_) > 3])] 
    dlemma.append(lm)
  dlemma

In [29]:
if treatment:
  tokens = []
  for doc in dlemma:
      tokens.append(doc[0].split())
  tokens

In [30]:
if treatment:
  phrases = Phrases(tokens, min_count=2)
  bigram = Phraser(phrases)
  bdocs = [bigram[d] for d in tokens]

  # [print(d) for d in bdocs]
  # for d in bdocs:
  #   for w in d:
  #     if '_' in w:
  #       print(w)

  tokens = bdocs

  with open(path+'tokens.txt', 'w') as temp_file:
    for row in tokens:
      for index, item in enumerate(row):
        if index == len(row)-1:
          temp_file.write("%s\n" % item)
        else:
          temp_file.write("%s," % item)

### Statistcs

- N of documents
- the shortest one (n of words)
- the longest one (n of words)
- the average size 

In [31]:
file = open(path+'tokens.txt', 'r')
lines = file.readlines()
tokens = [i.replace("\n", "").split(",") for i in lines]

n_documents = len(tokens)
shortest = min([len(d) for d in tokens])
largest = max([len(d) for d in tokens])
average = sum([len(d) for d in tokens]) / n_documents

print("number of documents: {}\nshortest: {}\nlargest: {}\naverage: {}".format(n_documents, shortest, largest, average))

number of documents: 1000
shortest: 31
largest: 514
average: 171.645


In [32]:
if wordcloud:
  coll = ''
  allTokens = []
  dictionaryOfString = []

  for d in tokens:
    [allTokens.append(w) for w in d]
    collect = ' '.join([w for w in d])
    dictionaryOfString.append(collect)
    coll = coll+' '+collect

  mycloud = wc.WordCloud().generate(coll)
  plt.figure(figsize=(20, 30))
  plt.imshow(mycloud)

  setOfTokens = set(allTokens)

  print("n of words of dictionary: {}\nn of words of collection: {}".format(
    len(setOfTokens), len(allTokens)))

# Part 2

Using the saved tokens as `corpus`

In [33]:
corpus = tokens

## Tomotopy

Using Tomotopy to get the topics

In [34]:
!pip install tomotopy
import tomotopy as tp
import numpy as np
ldamodel = tp.LDAModel

Using a type of gridSearch to get the best params (as the professor's example)

In [35]:
rm_top =[30] #the number of top words to be removed (default 0)
min_df = [0] #minimum document frequency of words (default 0)
min_cf = [0] #minimum collection frequency of words. (default 0)
alphas = [0.45] #hyperparameter of Dirichlet distribution for document-topic (default 0.1)
etas = [0.45]  #hyperparameter of Dirichlet distribution for topic-word (default 0.01)
K=[43]
if gridSearch:
    cv=[]
    print('# of docs:', len(corpus)) 
    for k in K:
      for mdf in min_df:
        for rm in rm_top:
          for a in alphas: 
            for e in etas:
              for mcf in min_cf:
                #create an object
                #tw term weight IDF (Inverse Document Frequency term weighting), ONE (equal - default), PMI (Use Pointwise Mutual Information term weighting)
                LDA = ldamodel(tw = tp.TermWeight.IDF, k=k, alpha = a , eta = e, seed = 1,  min_df = mdf, min_cf = mcf, rm_top = rm)
                #add documents to it
                for doc in corpus:
                    LDA.add_doc(doc)
                #train
                LDA.train(iter = 500) # iter (# of iterations - default 10) # workers (# of cores to be used - 0 all)
                #get the coherence (c_v)
                coh = tp.coherence.Coherence(LDA, coherence='c_v')
                average_coherence = coh.get_score()
                print('K: %2d mcf: %2d mdf: %2d rm: %2d alfa: %.3f beta: %.3f coherence: %.3f'%(k,mcf,mdf,rm,a,e,average_coherence),end=' Collection ')
                print(' Vocab size:', len(LDA.used_vocabs), ' # of words:', LDA.num_words)
                cv.append(average_coherence)

Creating the model and printing the topics

In [37]:
LDA = ldamodel(tw = tp.TermWeight.IDF, k=K[0], alpha = alphas[0] , eta = etas[0], seed = 1,  min_df = min_df[0], min_cf = min_cf[0], rm_top = rm_top[0])
#add documents to it
for doc in corpus:
    LDA.add_doc(doc)
#train
LDA.train(iter = 500) # iter (# of iterations - default 10) # workers (# of cores to be used - 0 all)

for i in range(K[0]):
    print("Topic", i, end=' => ')
    for w in LDA.get_topic_words(i):
        print(w[0], end=" ")
    print()

Topic 0 => masters nepal kathmandu hole woods augusta tiger_woods birdie golf quake 
Topic 1 => pylon lion gallipoli animal elephant jackson measle pool safechuck tomb 
Topic 2 => death student court friend officer girl report victim daughter wife 
Topic 3 => hatch rangers lynch kukucova tariq fifa trent ceglia heatlie gemini 
Topic 4 => hamilton race rosberg gazprom mercedes dunford racing grand_prix prix mccoy 
Topic 5 => pusok gray brain manyang santana drug depression ssri flakka device 
Topic 6 => toure napoli pardew palermo juventus lazio watford serie dybala ince 
Topic 7 => luke goff property bates water water_right secret_service albin hall garage 
Topic 8 => weinstein protein_world starbucks poster harry_potter qaeda recipe arousal bines chemical 
Topic 9 => indiana freedom teacher pupil hutchinson extremism arkansas chimpanzee huntsman value 
Topic 10 => bilby fairy_circle costello spacecraft sandstorm mason affleck mission asteroid mercury 
Topic 11 => mall lazarus wallenda

/tmp/ipykernel_63253/3175099011.py:6: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  LDA.train(iter = 500) # iter (# of iterations - default 10) # workers (# of cores to be used - 0 all)


There some topics that can be the same, for example topics of sports or crimes.